In [1]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
%matplotlib inline
plt.style.use('ggplot')

In [2]:
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout
style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [3]:
def ode_model(z, t, beta, kappa, gamma, delta): 
    S, E, I, R= z
    N = S + E + I + R 
    
    dSdt = -beta*S*I/N  + delta*R
    dEdt = beta*S*I/N - kappa*E
    dIdt = kappa*E - gamma*I
    dRdt = gamma*I - delta*R
    
    return [dSdt, dEdt, dIdt, dRdt] 

In [4]:
def ode_solver(t, initial_conditions, params):
    initE, initI, initR, initN = initial_conditions
    beta, kappa, gamma, delta = params 
    initS = initN - (initE + initI + initR)
    
    res = odeint(ode_model, [initS, initE, initI, initR], t, args=(beta, kappa, gamma, delta))
    return res

In [5]:
#NOTE: those initial conditions do not represent any physical meaning
initN = 2930000
initS = 2920098
initE = 1
initI = 1
initR = 0
beta = 1.52
delta = 0.04
kappa = 0.7
gamma = 0.62
days = 200

In [6]:
def main(initE, initI, initR, initN, beta, kappa, gamma, delta, days):
    initial_conditions = [initE, initI, initR, initN]
    params = [beta, kappa, gamma, delta]
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, initial_conditions, params)
    S, E, I, R = sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3]
    
    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=tspan, y=S, mode='lines+markers', name='Susceptible'))
    fig.add_trace(go.Scatter(x=tspan, y=E, mode='lines+markers', name='Exposed'))
    fig.add_trace(go.Scatter(x=tspan, y=I, mode='lines+markers', name='Infected'))
    fig.add_trace(go.Scatter(x=tspan, y=R, mode='lines+markers',name='Removed'))
    
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    
    fig.update_layout(title='Simulation of SEIRS Model',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=900, height=600,
                     )
    
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    fig.show()

In [7]:
interact(main, 
         initS=IntSlider(min=0, max=100000, step=1, value=initS, description='initS', style=style, layout=slider_layout),
         initE=IntSlider(min=0, max=100000, step=1, value=initE, description='initE', style=style, layout=slider_layout),
         initI=IntSlider(min=0, max=100000, step=10, value=initI, description='initI', style=style, layout=slider_layout),
         initR=IntSlider(min=0, max=100000, step=10, value=initR, description='initR', style=style, layout=slider_layout),
         initN=IntSlider(min=0, max=13800000, step=10, value=initN, description='initN', style=style, layout=slider_layout),
         beta=FloatSlider(min=0, max=2, step=0.01, value=beta, description='Infection rate', style=style, layout=slider_layout),
         kappa=FloatSlider(min=0, max=1, step=0.01, value=kappa, description='Exposure to Infection rate', style=style, layout=slider_layout), 
         gamma=FloatSlider(min=0, max=1, step=0.001, value=gamma, description='Removal Rate', style=style, layout=slider_layout),
         delta=FloatSlider(min=0, max=1, step=0.01, value=delta, description='Sus Feedback', style=style, layout=slider_layout),
         days=IntSlider(min=1, max=600, step=7, value=days, description='Days', style=style, layout=slider_layout)
        );

interactive(children=(IntSlider(value=1, description='initE', layout=Layout(width='99%'), max=100000, style=Sl…